In [1]:
import numpy as np
import pandas as pd 
import sklearn
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import sklearn.linear_model as lm

In [2]:
df = pd.read_csv('./train.csv')
df.fillna(0, inplace=True)
columns = ['price_doc', 'timestamp', 'full_sq', 'life_sq',
'floor', 'max_floor', 'material', 'build_year',
 'num_room', 'kitch_sq', 'state', 'product_type', 'sub_area']

df = df[columns]
df = df._get_numeric_data()

In [3]:
df_test = df.sample(n=1000, random_state=0)
df_train = df.drop(index=df_test.index)

df_test = df_test.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)

In [4]:
X_train = df_train[df_train.columns[1:]]
y_train = df_train['price_doc']

X_test = df_test[df_test.columns[1:]]
y_test = df_test['price_doc']

In [5]:
predictions = np.array([y_train.median()]*1000)

In [6]:
baseline = np.sqrt(mean_squared_log_error(y_test, predictions))
baseline
print(f"RMSLE все значения медиана:{baseline}")

RMSLE все значения медиана:0.6043387231257158


In [7]:

min_max = MinMaxScaler() 
min_max.fit(X_train)
X_train_min_max = min_max.transform(X_train)
X_test_min_max = min_max.transform(X_test)


standard = StandardScaler()
standard.fit(X_train)
X_train_standard = standard.transform(X_train)
X_test_standard = standard.transform(X_test)

In [8]:
linear_regression = lm.LinearRegression().fit(X=X_train_min_max, y=y_train)

In [9]:
#Предсказывает ,нахоидт метрику .
def calculate_metric(X_train, X_test, y_train, y_test, model):

    predictions_test = model.predict(X_test)
    predictions_train = model.predict(X_train)
    
    if (predictions_test < 0).any() or (predictions_train < 0).any(): 
        predictions_test[np.where(predictions_test < 0)] = 0
        predictions_train[np.where(predictions_train < 0)] = 0
    

    metric_test = np.sqrt(mean_squared_log_error( y_test, predictions_test))
    metric_train = np.sqrt(mean_squared_log_error( y_train, predictions_train))

    delta_baseline = baseline - metric_test
    return metric_test, metric_train

In [10]:
calculate_metric(X_train=X_train_min_max, X_test=X_test_min_max, y_train=y_train, y_test=y_test, model=linear_regression)

(np.float64(0.5588000012725938), np.float64(0.5571633028324698))

In [11]:
linear_regression2 = lm.LinearRegression().fit(X=X_train_standard, y=y_train)
metric_test, metric_train = calculate_metric(X_train=X_train_standard, X_test=X_test_standard, y_train=y_train, y_test=y_test, model=linear_regression2)

In [12]:
print("Предсказания для исходных данных:", linear_regression.predict(X_train_min_max[:5]))
print("Предсказания для стандартизированных данных:", linear_regression2.predict(X_train_standard[:5]))
#Почему то они одинаковые

Предсказания для исходных данных: [4956519.02066523 4473126.30577024 4734368.56425187 7240611.36643911
 6517931.6044505 ]
Предсказания для стандартизированных данных: [4956519.02066521 4473126.3057702  4734368.56425185 7240611.36643917
 6517931.60445054]


In [13]:
for c in df.columns[1:]:
    df[c+'_2'] = df[c]*df[c]
    df[c+'_3'] = df[c+'_2']*df[c]
    df[c+'_4'] = df[c+'_3']*df[c]

In [14]:
df

,price_doc,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,build_year_4,num_room_2,num_room_3,num_room_4,kitch_sq_2,kitch_sq_3,kitch_sq_4,state_2,state_3,state_4
0,5850000,43,27.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6000000,34,19.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5700000,43,29.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13100000,89,50.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16331452,77,77.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,7400000,44,27.0,7.0,9.0,1.0,1975.0,2.0,6.0,3.0,...,1.521488e+13,4.0,8.0,16.0,36.0,216.0,1296.0,9.0,27.0,81.0
30467,25000000,86,59.0,3.0,9.0,2.0,1935.0,4.0,10.0,3.0,...,1.401922e+13,16.0,64.0,256.0,100.0,1000.0,10000.0,9.0,27.0,81.0
30468,6970959,45,0.0,10.0,20.0,1.0,0.0,1.0,1.0,1.0,...,0.000000e+00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
30469,13500000,64,32.0,5.0,15.0,1.0,2003.0,2.0,11.0,2.0,...,1.609622e+13,4.0,8.0,16.0,121.0,1331.0,14641.0,4.0,8.0,16.0


In [15]:

df_test = df.sample(n=1000, random_state=0)
df_train = df.drop(index=df_test.index)

df_test = df_test.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)

In [16]:
X_train = df_train[df_train.columns[1:]]
y_train = df_train['price_doc']

X_test = df_test[df_test.columns[1:]]
y_test = df_test['price_doc']

In [17]:
min_max = MinMaxScaler()
min_max.fit(X_train)
X_train_min_max = min_max.transform(X_train)
X_test_min_max = min_max.transform(X_test)

standard = StandardScaler()
standard.fit(X_train)
X_train_standard = standard.transform(X_train)
X_test_standard = standard.transform(X_test)

In [18]:
linear_regression = lm.LinearRegression().fit(X=X_train_min_max, y=y_train)

In [19]:
calculate_metric(X_train=X_train_min_max, X_test=X_test_min_max, y_train=y_train, y_test=y_test, model=linear_regression)

(np.float64(0.5320992241016059), np.float64(0.6420427413553875))

In [20]:
ridge = lm.Ridge().fit(X=X_train_min_max, y=y_train)

In [21]:
linear_regression.coef_

array([ 4.53747223e+08,  2.01498451e+08,  3.01262888e+07, -1.07526692e+07,
        7.22479477e+06,  3.97608801e+10, -3.91256172e+07,  5.30913892e+07,
       -8.53261888e+07,  8.97022013e+09, -1.14086870e+11,  1.04668618e+11,
       -9.61940688e+09,  8.16782423e+10, -7.22620446e+10, -2.24679828e+08,
        6.15662926e+08, -4.16197543e+08,  8.72233621e+07, -1.68387209e+08,
        9.42678314e+07,  1.21497670e+07, -4.29250321e+07,  2.47650672e+07,
       -3.37560774e+14,  1.68595830e+14,  1.68767235e+14,  2.39975146e+08,
       -1.89619783e+08, -1.97025568e+07, -3.36176982e+08,  5.86057797e+08,
       -3.05875318e+08,  3.54644389e+09, -4.57814453e+10,  2.00273671e+11])

In [22]:
calculate_metric(X_train=X_train_min_max, X_test=X_test_min_max, y_train=y_train, y_test=y_test, model=ridge)

(np.float64(0.5543492717965641), np.float64(0.5544899453593611))

In [23]:
lasso = lm.Lasso().fit(X=X_train_min_max, y=y_train)

C:\Users\Robot\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.962e+17, tolerance: 6.758e+13
  model = cd_fast.enet_coordinate_descent(


In [24]:
calculate_metric(X_train=X_train_min_max, X_test=X_test_min_max, y_train=y_train, y_test=y_test, model=lasso)

(np.float64(0.5319535894961515), np.float64(0.6350687703682347))

In [25]:

lasso = lm.Lasso(alpha=10).fit(X=X_train_min_max, y=y_train)

coefficients = pd.Series(lasso.coef_, index=X_train.columns)

# Выводим признаки, у которых коэффициент = 0
features_to_remove = coefficients[coefficients == 0].index.tolist()
print("Признаки, которые можно удалить:", features_to_remove)


Признаки, которые можно удалить: ['build_year', 'life_sq_3', 'life_sq_4', 'num_room_4']


C:\Users\Robot\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.730e+17, tolerance: 6.758e+13
  model = cd_fast.enet_coordinate_descent(
